# CLIP Image Search
Search and rank a folder of images using a text prompt.

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install torchvision

In [ ]:
import os
import clip
import torch
from PIL import Image
import IPython.display as display

In [ ]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# Upload images
from google.colab import files
uploaded = files.upload()
image_filenames = list(uploaded.keys())

In [ ]:
# Encode images
images = []
image_inputs = []
for filename in image_filenames:
    image = preprocess(Image.open(filename)).unsqueeze(0).to(device)
    images.append(filename)
    image_inputs.append(image)

image_input = torch.cat(image_inputs, dim=0)
with torch.no_grad():
    image_features = model.encode_image(image_input)

In [ ]:
# Search
query = input("Enter a text description: ")
text_input = clip.tokenize([query]).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_input)

similarity = (text_features @ image_features.T).squeeze(0)
values, indices = similarity.topk(5)

print("\nTop Matches:")
for i in indices:
    display.display(Image.open(images[i]))
    print(f"Image: {images[i]} | Score: {similarity[i].item():.2f}")